# Task 1: Binary Representations

In this section, It creates some basic bit manipulation functions.
Trying to rotate bits to the left

References from https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

# rotl(x,n=1), rotating left that rotates the bits in a 32-bit unsigned integer to the left n places.



In [13]:
# First try at rotate left... but I missed something

def rotl(x, n=1):
    return (x << n) | (x >> n)

# Testing with a simple number that only has 1 bit on
print(bin(rotl(0b00000000000000000000000000000001, 1)))


0b10


does not rotate correctly.
shifted right by n, but to wrap the bits properly I should shift right by 32 - n. Also, forgot to make sure we stay within 32 bits .

In [ ]:
# Fixed it by wrapping the bits properly and keeping the result to 32 bits.
# ref on https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

def rotatel(x, n=1):

    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

# Testing again
print(bin(rotatel(0b00000000000000000000000000000001, 1)))  
print(bin(rotatel(0x80000000, 1)))  


0b10
0b1


# rotr (x,n=1) that rotates the bits in a 32-bit unsigned integer to the right n places.





this is basically the opposite of the last one.
rotating bits to the right, and wrapping it back to the left

ref : https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

In [ ]:
# First attempt at rotating right – but something feels off

def rotater(x, n=1):
    return (x >> n) | (x << n)

print(bin(rotater(0b00000000000000000000000000000010, 1)))  # expecting 0b1


0b101


In [22]:
# Fixed it 
def rotater(x, n=1):
   
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

print(bin(rotater(0b10, 1)))   
print(bin(rotater(0b1, 1)))    


0b1
0b10000000000000000000000000000000



the `return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF` function spins the bits to the right by n steps making sure everything stays within 32 bits  see https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb sections: bitwise shift , bitwise OR , bit masking , integer size

## ch ( x, y, z) that chooses the bits from y where x has bits set to 1 and bits in z where x has bits set to 0.

This function is like a little decision-maker.  
It looks at each bit of `x` — if it’s a 1, it picks the corresponding bit from `y`. If it’s a 0, it picks from `z`.

In [6]:
def ch(x, y, z):
    return (x & y) | (x & z)

print(bin(ch(0b1010, 0b1100, 0b0011))) # expecting 0b1010
print(bin(ch(0b1010, 0b1100, 0b0000))) # expecting 0b1000
print(bin(ch(0b1010, 0b0000, 0b0011))) # expecting 0b0000


0b1010
0b1000
0b10


In [7]:
# 
def ch(x, y, z):
   
    return (x & y) ^ (~x & z)

print(bin(ch(0b1010, 0b1100, 0b0011)))  # expecting 0b1010
print(bin(ch(0b1010, 0b1100, 0b0000)))  # expecting 0b1000
print(bin(ch(0b1010, 0b0000, 0b0011)))  # expecting 0b0000
print(bin(ch(0b1010, 0b1100, 0b1111)))  # expecting 0b1111

0b1001
0b1000
0b1
0b1101


The ch(x, y, z) function is based on bitwise operations covered in the Bitwise AND, OR, XOR, and NOT sections see ref: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb combining them to make a per-bit logic - "if x then y else z"

## maj(x, y, z)  which takes a majority vote of the bits in x, y, and z.
The output should have a 1 in bit position i where at least two of x, y, and z have 1's in position i.
All other output bit positions should be 0.

This function is used in hashing algorithms to "vote" on each bit.
For each bit position `i`, it returns 1 if at least **two** of `x`, `y`, and `z` have a 1 in that position.

Example:  
- x = 1010  
- y = 1111  
- z = 0000  
→ maj = 1010 (only positions where at least two are 1)


In [ ]:
def maj(x, y, z): # majority function
    
    return x | y | z 

# Testing 
print(bin(maj(0b1010, 0b1100, 0b0011))) 


0b1111


In [ ]:
def maj(x, y, z):
    # Majority function: returns 1 if two or more of the bits are 1
    # and 0 otherwise.
    return (x & y) ^ (x & z) ^ (y & z) 

# Testing a few values:
x = 0b10101010 
y = 0b11110000
z = 0b00001111

print("x:", format(x, '08b')) 
print("y:", format(y, '08b'))
print("z:", format(z, '08b'))
print("maj:", format(maj(x, y, z), '08b'))


x: 10101010
y: 11110000
z: 00001111
maj: 10101010


### How the maj(x, y, z) logic works

The formula `(x & y) ^ (x & z) ^ (y & z)` works because it captures all the cases where **two or more** inputs have a 1.


- `x & y` → 1 only if both x and y have 1
- `x & z` → 1 if x and z do
- `y & z` → 1 if y and z do
Then XOR-ing all three gives us 1 in any position where **at least two** inputs have 1s.

results based on Bitwise (&) and XOR referenced in https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb 

- bitwise AND : talks how xx & y returns 1 only when both bits are 1.
- bitwise XOR : usefull when combining partial matches lin ( x & y ) ^ ( x & z) ^ (y & z).


# Task 2: Hash Functions

translation of a hash function from C into Python.



In [2]:
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}


SyntaxError: invalid syntax (2215498579.py, line 1)

the goals is to 
- Convert the function to Python.

- Test it with a few strings.

- Explain why 31 and 101 are used.

- Include mistake version first, then correct version.

- Add natural markdown, comments, and commits.

In [13]:
def unsigned_hash(s):
    # Simple hash function that takes a string and returns a hash value.
    hashval = 0 # Initialize hash value to 0    
    for char in s: # Iterate over each character in the string
        hashval = hashval * 31 + ord(s) # Update hash value using the character's ASCII value
    return hashval % 101 # Return the hash value modulo 101 to keep it within a reasonable range
# Testing the simple_hash function
print(unsigned_hash("hello"))  


TypeError: ord() expected a character, but string of length 5 found

In [14]:
def unsigned_hash(s):
    
    hashval = 0 # Initialize hash value to 0
    for char in s: # Iterate over each character in the string 
        hashval = ord(char) + 31 * hashval # Update hash value using the character's ASCII value
    return hashval % 101 # Return the hash value modulo 101 to keep it within a reasonable range

# Testing the simple_hash function with various strings
print("Hash of 'hello':", unsigned_hash("hello"))
print("Hash of 'abc':", unsigned_hash("abc"))
print("Hash of 'hashing':", unsigned_hash("hashing"))
print("Hash of 'function':", unsigned_hash("function"))
print("Hash of 'test':", unsigned_hash("test"))
print("Hash of 'example':", unsigned_hash("example"))
print("Hash of 'data':", unsigned_hash("data"))


Hash of 'hello': 17
Hash of 'abc': 0
Hash of 'hashing': 25
Hash of 'function': 100
Hash of 'test': 86
Hash of 'example': 28
Hash of 'data': 55


## Conluding

## Why 31 and 101?

The unsigned_hash(s) function uses the logic described in see reference: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/prime_numbers.ipynb 
- it multiplies by 31 (a small prime that can be computed efficiently using bit shifting) and uses modulo 101 (another prime) to keep the hash within a fixed range. The use of bitwise shift (<<) as a fast multiplication method see reference : https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

# Task 3: SHA256 

Write a Python function that calculates the SHA256 padding for a given file.

- The function should take a file path as input.
- It should print, in hex, the padding that would be applied to it.
- The specification states that the following should be appended to a message:

- a 1 bit;
- enough 0 bits so the length in bits of padded message is the smallest possible multiple of 512;
- the length in bits of the original input as a big-endian 64-bit unsigned integer.

The example in the specification is a file containing the three bytes abc:

01100001 01100010 01100011

The output would be:

80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 18


In [107]:
def sha256_padding(file_path):
    with open(file_path, "rb") as f:
        data = f.read()

    bit_len = len(data) * 8
    padding = b'\x80'

    
    while (len(data) + len(padding) + 8) % 512 != 0:
        padding += b'\x00'

    bit_len_bytes = bit_len.to_bytes(8, 'big')
    padded = data + padding + bit_len_bytes

    print("Padded message (hex):", padded.hex())


In [139]:
def sha256_padding(file_path):

    with open(file_path, "rb") as f:
        data = f.read()

    bit_len = len(data) * 8  # total message length in bits
    padding = b'\x80'        # first padding byte: 10000000

    # Add 0x00 bytes until length ≡ 56 mod 64
    while (len(data) + len(padding) + 8) % 64 != 0:
        padding += b'\x00'

    # 64-bit big-endian length
    bit_len_bytes = bit_len.to_bytes(8, 'big')

    padded_msg = data + padding + bit_len_bytes
    full_padding = padding + bit_len_bytes

    print("Original data (hex):", data.hex())
    print("Padding (hex):", padding.hex())
    print("Length bytes (hex):", bit_len_bytes.hex())
    print("Full padded message (hex):", padded_msg.hex())
    print("Padded message length (bytes):", len(padded_msg))
    print("Padded message length (bits):", len(padded_msg) * 8)
    print("\nPadding Output:")
    print(" ".join(f"{byte:02x}" for byte in full_padding))



In [140]:
sha256_padding("abc.txt")


Original data (hex): 616263
Padding (hex): 8000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Length bytes (hex): 0000000000000018
Full padded message (hex): 61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
Padded message length (bytes): 64
Padded message length (bits): 512

Padding Output:
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


The sha256_padding() function implements the SHA-256 message padding procedure mentioned in Motivation under Binary Representations (FIPS PUB 180-4 see reference: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb) . It uses concepts such as bit and byte length calculation, hexadecimal formatting, and big-endian encoding ,everything needed  for preparing the message block for SHA-256 processing.

# Task 4: Prime Numbers

Calculate the first 100 prime numbers using two different algorithms.
Any algorithms that are well-established and works correctly are okay to use.
Explain how the algorithms work.

In [ ]:
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):  
        if n % i == 0:
            return False
    return True

print(is_prime(2))
print(is_prime(1))
print(is_prime(0))

def first_100_primes_trial():
    primes = []
    num = 0  # starts from 0
    while len(primes) <= 100:
        if is_prime(num):
            primes.append(num)
        num += 1
    return primes

print(first_100_primes_trial())


True
False
False
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547]
